In [18]:
import os
import sys 
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

# Directories
nb_dir = Path.cwd()
REPO_ROOT = nb_dir.parent
data_dir = REPO_ROOT / 'data/'
processed_dir = data_dir / 'processed/'
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

In [30]:
# Avalanche Canada polygons (GeoJSON)
avcan_path = REPO_ROOT / "data/external/avalanche_canada/canadian_subregions.geojson"
avcan = gpd.read_file(avcan_path)

# NBAC / BC fire perimeters (your processed shapefile or geojson)
fires_path = REPO_ROOT / "data/processed/BC_fires/BC_fires_2024_shp/BC_fires_2024_shp.shp"
bcfires = gpd.read_file(fires_path)

print(f"Avalanche Canada Regions Loaded. {avcan.crs}")
print(f"British Columbia Fires Loaded. {bcfires.crs}")

Avalanche Canada Regions Loaded. EPSG:4326
British Columbia Fires Loaded. EPSG:4326


In [34]:
centroids = bcfires.geometry.centroid

/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_76864/1110416210.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = bcfires.geometry.centroid


In [35]:
centroids

0       POINT (-121.69824 58.21446)
1       POINT (-121.27247 59.07098)
2       POINT (-115.45529 49.91415)
3       POINT (-121.40311 50.52599)
4       POINT (-121.33948 52.65509)
                   ...             
2748    POINT (-121.09001 51.48641)
2749    POINT (-118.57281 51.71992)
2750    POINT (-118.40535 51.83428)
2751    POINT (-118.17372 52.14149)
2752    POINT (-123.76698 57.61115)
Length: 2753, dtype: geometry

In [43]:
avcan["reference_region"].value_counts()

reference_region
South Coast Inland      9
Jasper                  9
South Columbia          9
Banff Yoho Kootenay     7
North Rockies           7
Northwest Coastal       7
Northwest Inland        7
Kootenay Boundary       7
Sea To Sky              6
Cariboos                6
Purcells                6
North Columbia          6
South Rockies           5
South Coast             4
Vancouver Island        4
Yukon                   4
Kananaskis              4
Chic-Chocs              3
Long Range Mountains    3
Lizard-Flathead         3
Little Yoho             2
Glacier                 1
Waterton Lakes          1
Name: count, dtype: int64

In [45]:
# Keep only what you need from avcan to avoid duplicate columns
regions = avcan[["reference_region","polygon_name", "geometry"]]   # adjust column names as needed

fires_with_region = gpd.sjoin(
    bcfires,
    regions,
    how="inner",          # drop fires that don't hit any region
    predicate="intersects"  # or "within" if you use centroids/points
)

fires_with_region.head()


,GID,NFIREID,YEAR,ADMIN_AREA,ADJ_HA,geometry,index_right,reference_region,polygon_name
2,2024_161,161.0,2024.0,BC,10860.260253,"MULTIPOLYGON (((-115.50191 49.92091, -115.5020...",80,South Rockies,Bull
4,2024_163,163.0,2024.0,BC,1613.418689,"MULTIPOLYGON (((-121.26353 52.62446, -121.2637...",4,Cariboos,Quesnel
6,2024_165,165.0,2024.0,BC,152.502534,"POLYGON ((-121.55579 52.87224, -121.55622 52.8...",4,Cariboos,Quesnel
7,2024_166,166.0,2024.0,BC,11413.094438,"MULTIPOLYGON (((-121.45826 52.97016, -121.4579...",4,Cariboos,Quesnel
8,2024_167,167.0,2024.0,BC,333.129129,"POLYGON ((-121.11494 53.11394, -121.11434 53.1...",2,Cariboos,McBride


In [37]:
fires_with_region.shape

(1362, 8)

In [47]:
fires_by_region = (
    fires_with_region
    .groupby(["reference_region","polygon_name"])
    .agg(
        n_fires=("NFIREID", "nunique"),
        total_area_ha=("ADJ_HA", "sum")
    )
    .reset_index()
)
fires_by_region.sort_values("n_fires")

,reference_region,polygon_name,n_fires,total_area_ha
27,North Rockies,Robson,1,1.099315
23,North Rockies,East Kakwa,1,25.762692
34,Northwest Coastal,Stewart,1,8.805966
35,Northwest Inland,Howson,1,841.622180
6,Glacier,Glacier,1,779.183544
...,...,...,...,...
22,North Columbia,Shuswap,53,57552.560107
12,Kootenay Boundary,South Okanagan,57,42791.257877
1,Cariboos,Clearwater,57,24592.538118
67,South Columbia,Gold,62,19967.303920


Option 2 – clip fires to all regions at once

In [58]:
fires_clipped = gpd.clip(bcfires, regions)

print(fires_clipped.shape)
fires_clipped.head()

(1324, 6)


,GID,NFIREID,YEAR,ADMIN_AREA,ADJ_HA,geometry
373,2024_1279,1279.0,2024.0,BC,2.340934,"POLYGON ((-116.23318 49.00226, -116.23259 49.0..."
1742,2018_1124,1124.0,2018.0,BC,34.546910,"POLYGON ((-116.08536 49.00181, -116.08521 49.0..."
1845,2018_1779,1779.0,2018.0,BC,1012.394746,"POLYGON ((-115.89199 49.02578, -115.89087 49.0..."
1365,2022_793,793.0,2022.0,BC,1087.988304,"MULTIPOLYGON (((-114.64019 49.00131, -114.6401..."
328,2024_1234,1234.0,2024.0,BC,3.624654,"POLYGON ((-115.66814 49.01671, -115.66810 49.0..."


In [59]:
fires_clipped = gpd.sjoin(
    fires_clipped,
    regions,
    how="inner",
    predicate="within"
)

In [60]:
fires_clipped

,GID,NFIREID,YEAR,ADMIN_AREA,ADJ_HA,geometry,index_right,reference_region,polygon_name
373,2024_1279,1279.0,2024.0,BC,2.340934,"POLYGON ((-116.23318 49.00226, -116.23259 49.0...",49,Purcells,Moyie
1845,2018_1779,1779.0,2018.0,BC,1012.394746,"POLYGON ((-115.89199 49.02578, -115.89087 49.0...",49,Purcells,Moyie
1365,2022_793,793.0,2022.0,BC,1087.988304,"MULTIPOLYGON (((-114.64019 49.00131, -114.6401...",14,Lizard-Flathead,Flathead
328,2024_1234,1234.0,2024.0,BC,3.624654,"POLYGON ((-115.66814 49.01671, -115.66810 49.0...",49,Purcells,Moyie
1459,2019_807,807.0,2019.0,BC,50.973860,"POLYGON ((-115.34939 49.02598, -115.34915 49.0...",49,Purcells,Moyie
...,...,...,...,...,...,...,...,...,...
2186,2018_2493,2493.0,2018.0,BC,13.981216,"POLYGON ((-130.23427 57.50010, -130.23412 57.4...",42,Northwest Inland,Ningunsaw
2187,2018_2494,2494.0,2018.0,BC,5.066133,"POLYGON ((-130.00302 57.67027, -130.00342 57.6...",42,Northwest Inland,Ningunsaw
1668,2018_584,584.0,2018.0,BC,7919.508929,"MULTIPOLYGON (((-130.89975 57.58994, -130.9001...",42,Northwest Inland,Ningunsaw
1626,2018_274,274.0,2018.0,BC,3625.041221,"POLYGON ((-130.68109 57.94028, -130.68042 57.9...",42,Northwest Inland,Ningunsaw


In [50]:
fires_regions = gpd.overlay(
    bcfires,
    regions,
    how="intersection"   # intersection of fire and region polygons
)

fires_regions.head()

,GID,NFIREID,YEAR,ADMIN_AREA,ADJ_HA,reference_region,polygon_name,geometry
0,2024_161,161.0,2024.0,BC,10860.260253,South Rockies,Bull,"MULTIPOLYGON (((-115.39383 49.90509, -115.3938..."
1,2024_163,163.0,2024.0,BC,1613.418689,Cariboos,Quesnel,"MULTIPOLYGON (((-121.22784 52.66976, -121.2284..."
2,2024_165,165.0,2024.0,BC,152.502534,Cariboos,Quesnel,"POLYGON ((-121.55622 52.87219, -121.55763 52.8..."
3,2024_166,166.0,2024.0,BC,11413.094438,Cariboos,Quesnel,"MULTIPOLYGON (((-121.23632 53.01775, -121.2360..."
4,2024_167,167.0,2024.0,BC,333.129129,Cariboos,McBride,"POLYGON ((-121.11434 53.11387, -121.11324 53.1..."


In [69]:
fires_regions[fires_regions['GID']=="2021_1194"]

,GID,NFIREID,YEAR,ADMIN_AREA,ADJ_HA,reference_region,polygon_name,geometry,area_ha
1134,2021_1194,1194.0,2021.0,BC,2409.88022,North Rockies,Pine Pass,"POLYGON ((-1639888.851 1085087.163, -1639888.8...",2490.180328
1135,2021_1194,1194.0,2021.0,BC,2409.88022,North Rockies,Tumbler,"POLYGON ((-1634826.421 1084417.487, -1634878.0...",0.518555


In [65]:
fires_regions['NFIREID'].value_counts()

NFIREID
1194.0    4
1233.0    4
1560.0    3
1302.0    3
747.0     3
         ..
553.0     1
554.0     1
555.0     1
556.0     1
2746.0    1
Name: count, Length: 1146, dtype: int64

In [51]:
fires_regions.shape

(1362, 8)

In [78]:
# Make sure we use a projected CRS in metres for area calc
if fires_regions.crs.is_geographic:  # e.g. EPSG:4326
    fires_regions = fires_regions.to_crs("EPSG:3978")  # Canada Lambert as an example

fires_regions["area_ha"] = fires_regions.geometry.area / 10_000

burned_by_region = (
    fires_regions
    .groupby(["reference_region","polygon_name"])
    .agg(
        n_fires=("GID", "nunique"),             # GID = year + fire id, unique per fire event
        burned_ha=("area_ha", "sum")
    )
    .reset_index()
)

In [79]:
burned_by_region.sort_values('burned_ha')

,reference_region,polygon_name,n_fires,burned_ha
27,North Rockies,Robson,1,0.845919
6,Glacier,Glacier,1,1.220156
40,Northwest Inland,Telkwa,2,4.629380
52,South Coast,North Shore,2,5.935566
34,Northwest Coastal,Stewart,1,7.255262
...,...,...,...,...
62,South Coast Inland,Stein,28,41959.445069
12,Kootenay Boundary,South Okanagan,57,43030.793185
22,North Columbia,Shuswap,53,44583.374623
59,South Coast Inland,Manning,16,48750.750149
